In [55]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime

## looking into data

In [65]:
tweets_df = pd.read_csv("dataset/javascript_top.csv")
tweets_df.head().T

,0,1,2,3,4
body,"""Return [data, loading, error];}"" http://socia...","Late tweet, 5th 6th (Days 53, 54). Very little...",Vue.js 2 Essentials: Build Your First Vue App☞...,When u write 500 lines of codes and exit witho...,My #first chat app hope to collaborate #javasc...
likes,0,0,0,2,NaN
link,/eibrahim/status/1093513673618964480,/L1K3R0535/status/1093513800098242565,/javascript_devv/status/1093514298486341632,/BarineSambaris/status/1092896518573568008,/GoodwishSifiso/status/1093224111919251456
replies,0,0,0,3,NaN
retweets,NaN,3,0,0,0
time,6:15 AM - 7 Feb 2019,6:16 AM - 7 Feb 2019,6:18 AM - 7 Feb 2019,1:23 PM - 5 Feb 2019,11:05 AM - 6 Feb 2019
writer,@eibrahim,@L1K3R0535,@javascript_devv,@BarineSambaris,@GoodwishSifiso


In [66]:
tweets_df.isna().sum()

body          0
likes        17
link          0
replies     172
retweets     40
time          0
writer        0
dtype: int64

### we found that all missing values are in fact  0 so we change them to 0

In [67]:
tweets_df.replies.fillna(0,inplace=True)
tweets_df.retweets.fillna(0,inplace=True)
tweets_df.likes.fillna(0,inplace=True)

### now we will convert our time feature to 3 features day and month and year

In [68]:
months = {'jan':1, 'feb':2, 'mar':3, 'apr':4,
          'may':5, 'jun':6, 'jul':7, 'aug':8,
          'sep':9, 'oct':10, 'nov':11, 'dec':12}

def get_time(date_time):
    return date_time.split('-')[0].strip()

def get_day(date_time):
    return int(date_time.split('-')[1].strip().split(' ')[0].strip())

def get_month(date_time):
    return months[date_time.split('-')[1].strip().split(' ')[1].strip().lower()]

def get_year(date_time):
    return int(date_time.split('-')[1].strip().split(' ')[2].strip())

In [69]:
def get_time_column(df, column_name):
    date_list = []
    column = df[column_name]
    for item in column:
        time = get_time(item)
        day = get_day(item)
        month = get_month(item)
        year = get_year(item)
        date_list.append(f'{time} - {day}/{month}/{year}')
    return pd.to_datetime(date_list, format='%I:%M %p - %d/%m/%Y')
tweets_df['time'] = get_time_column(tweets_df, 'time')
tweets_df['hour'] = tweets_df.time.dt.hour 

## Content exploration per period

In [78]:
def get_top_content(df, periods):
    top_content = []
    df['time-period'] = df.hour//4
    for period in range(periods):
        top_content.append(df[df['time-period'] == period])
    df.drop(columns=['time-period'], inplace=True)
    return top_content

In [79]:
x = get_top_content(tweets_df, 6)

In [89]:
x[0].hour.unique()

array([1, 2, 0, 3])